# Aadhaar 360: National Strategic Intelligence Grid
## Notebook 2: The 5 Strategic Solutions

This notebook implements the exact 5 solutions defined in the strategic vision:
1.  **👻 The Ghost Child Finder**: Comparing Child Enrolments to expected baselines.
2.  **🚧 The Border Watch**: 18+ Enrolment Tripwire (National Security).
3.  **📡 The Migration Radar**: Detecting Rural -> Urban shifts via Demographic Updates.
4.  **🛡️ The Integrity Shield**: Volume Anomaly Engine for fraud detection.
5.  **🎓 Youth Awareness Tracker**: Finding 15-year-olds missing mandatory biometric updates.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set Aesthetics
sns.set_theme(style="whitegrid", palette="rocket")
plt.rcParams['figure.figsize'] = (14, 8)

# Load Preprocessed Data
try:
    enrolment = pd.read_csv('d:/UIDAI/processed_data/enrolment_clean.csv', parse_dates=['date'])
    demographic = pd.read_csv('d:/UIDAI/processed_data/demographic_clean.csv', parse_dates=['date'])
    biometric = pd.read_csv('d:/UIDAI/processed_data/biometric_clean.csv', parse_dates=['date'])
    print("Data Loaded Successfully.")
except FileNotFoundError:
    print("Error: Preprocessed data not found. Run Notebook 1 first.")

## 1. 👻 The "Ghost Child" Finder
**Logic**: "We assume all children are enrolled, but thousands are missing."
**Method**: We identify districts where Child Enrolment (0-5) is suspiciously low compared to the total population activity.

*Alert triggered if Child Enrolment % < National Average - 1 StdDev*

In [ ]:
district_ages = enrolment.groupby(['state', 'district'])[['age_0_5', 'age_5_17', 'age_18_greater']].sum().reset_index()
district_ages['Total'] = district_ages[['age_0_5', 'age_5_17', 'age_18_greater']].sum(axis=1)
district_ages['Child_Share'] = district_ages['age_0_5'] / district_ages['Total']

# Threshold: Districts dropping below acceptable child enrolment rates
threshold = district_ages['Child_Share'].mean() - district_ages['Child_Share'].std()
ghost_child_districts = district_ages[district_ages['Child_Share'] < threshold].sort_values('Child_Share')

print(f"ALERT: Found {len(ghost_child_districts)} districts with potential 'Ghost Children' (Low Enrolment).")
display(ghost_child_districts[['state', 'district', 'age_0_5', 'age_18_greater']].head(5))

# Viz
plt.figure(figsize=(10, 6))
sns.regplot(data=district_ages, x='age_18_greater', y='age_0_5', scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title('Ghost Child Finder: Child vs Adult Enrolments (Points below line are Gaps)')
plt.show()

## 2. 🚧 The Border Watch (National Security)
**Logic**: "For 18+ enrollments, the count should ideally be zero."
**Method**: Any district reporting significant `age_18_greater` enrolments is flagged for immediate STF verification.

In [ ]:
# Filter for Adult Enrolments > 0
adult_enrolments = enrolment[enrolment['age_18_greater'] > 0].copy()
adult_alerts = adult_enrolments.groupby(['state', 'district'])['age_18_greater'].sum().reset_index().sort_values('age_18_greater', ascending=False)

print("SECURITY ALERT: Top Districts with Adult Enrolments (Need Verification):")
display(adult_alerts.head(10))

# Heatmap of risk
plt.figure(figsize=(12, 6))
if not adult_alerts.empty:
    sns.barplot(data=adult_alerts.head(10), x='age_18_greater', y='district', palette='Reds_r')
    plt.title('Border Watch: Districts with Highest 18+ New Enrolments')
plt.show()

## 3. 📡 The Migration Radar
**Logic**: "Villages empty out and Cities get overcrowded."
**Method**: High volume of *Demographic Updates* (Address changes) signals a Migration Hub.

In [ ]:
migration_data = demographic.groupby(['state', 'district'])['Total_Demo_Updates'].sum().reset_index()
top_migration_hubs = migration_data.sort_values('Total_Demo_Updates', ascending=False).head(10)

print("Top Destination Districts (Migration Hubs):")
display(top_migration_hubs)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_migration_hubs, x='Total_Demo_Updates', y='district', palette='Blues_r')
plt.title('Migration Radar: Highest Volume of Demographic Updates')
plt.show()

## 4. 🛡️ The Integrity Shield (Universal Vigilance)
**Logic**: "Sudden 100 enrolments instead of 10."
**Method**: Isolation Forest to detect throughput anomalies.

In [ ]:
from sklearn.ensemble import IsolationForest

daily_ops = enrolment.groupby(['date', 'district'])['Total_Enrolments'].sum().reset_index()

# Train Anomaly Detector
model = IsolationForest(contamination=0.005, random_state=42) # Strict 0.5% anomalies
daily_ops['anomaly'] = model.fit_predict(daily_ops[['Total_Enrolments']])

anomalies = daily_ops[daily_ops['anomaly'] == -1]
print(f"INTEGRITY ALERT: Detected {len(anomalies)} suspicious spikes.")

plt.figure(figsize=(14, 6))
plt.scatter(daily_ops['date'], daily_ops['Total_Enrolments'], alpha=0.3, label='Normal Flow')
plt.scatter(anomalies['date'], anomalies['Total_Enrolments'], color='red', label='Anomalies (Spikes)', s=50)
plt.title('Integrity Shield: Operational Anomalies')
plt.legend()
plt.show()

## 5. 🎓 Youth Awareness Tracker
**Logic**: "Teenagers (Age 15) are forgetting Mandatory Biometric Updates."
**Method**: Identify districts with low `bio_age_5_17` activity relative to their projected youth population.
*Proxy: We use the ratio of 5-17 Biometric Updates to 5-17 Enrolments.*

In [ ]:
youth_bio = biometric.groupby(['state', 'district'])['bio_age_5_17'].sum().reset_index()
youth_enrol = enrolment.groupby(['state', 'district'])['age_5_17'].sum().reset_index()

youth_metrics = pd.merge(youth_bio, youth_enrol, on=['state', 'district'])

# Update Ratio: How many updates per new enrolment? (Low ratio = High Gap)
youth_metrics['Update_Ratio'] = youth_metrics['bio_age_5_17'] / (youth_metrics['age_5_17'] + 1)

lagging_districts = youth_metrics.sort_values('Update_Ratio').head(10)

print("AWARENESS GAP: Top 10 Districts needing Youth Bio-Update Camps:")
display(lagging_districts[['state', 'district', 'bio_age_5_17', 'Update_Ratio']])